In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [4]:
# bs=48
bs=128

In [5]:
path = f'/home/kuptservol/.kaggle/datasets/jigsaw-toxic-comment-classification-challenge'

In [46]:
! ls {path}

data_clas   imdb_textlist_class_toxic  sample_submission.csv  train.csv
data_lm     lm_databunch	       test.csv
export.pkl  models		       test_labels.csv


In [7]:
train = pd.read_csv(f'{path}/train.csv')
test = pd.read_csv(f'{path}/test.csv')
subm = pd.read_csv(f'{path}/sample_submission.csv')

In [8]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [9]:
train['comment_text'][19]

"Don't mean to bother you \n\nI see that you're writing something regarding removing anything posted here and if you do oh well but if not and you can acctually discuss this with me then even better.\n\nI'd like to ask you to take a closer look at the Premature wrestling deaths catagory and the men listed in it, surely these men belong together in some catagory. Is there anything that you think we can do with the catagory besides delting it?"

In [10]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805,0.898321
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420,0.302226
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [12]:
len(train),len(test)

(159571, 153164)

## LSTM pretrain encoder

In [18]:
from fastai.text import *

In [19]:
bs=64

In [20]:
df = pd.concat([train,test], sort=False)

In [22]:
data_lm = (TextList.from_df(df, path, cols=COMMENT)
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [23]:
data_lm.save('data_lm')

In [15]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=1.)

In [16]:
lr = 1e-2
lr *= bs/48

In [17]:
learn_lm.fit_one_cycle(1, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.421038,3.951673,0.311978,12:56


In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(3, lr/10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.779420,3.636185,0.356762,14:39
1,3.706732,3.514719,0.373025,14:40
2,3.650321,3.468143,0.378759,14:41


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
learn_lm.save('fine_tuned_10')
learn_lm.save_encoder('fine_tuned_enc_10')

## Multi-label classifier

In [25]:
data_clas = (TextList.from_df(train, path, vocab=data_lm.vocab, cols=COMMENT)
        .split_by_rand_pct(0.1, seed=42)
        .label_from_df(cols=label_cols)
        .databunch(bs=bs, num_workers=1))

In [26]:
data_class.save('data_clas')

In [28]:
learn = text_classifier_learner(data_class, AWD_LSTM, drop_mult=0.5)

In [29]:
learn.load_encoder('fine_tuned_enc_10')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (143614 items)
x: TextList
xxbos xxmaj explanation 
  xxmaj why the edits made under my username xxmaj hardcore xxmaj metallica xxmaj fan were reverted ? xxmaj they were n't vandalisms , just closure on some gas after i voted at xxmaj new xxmaj york xxmaj dolls xxup fac . xxmaj and please do n't remove the template from the talk page since i 'm retired xxunk,xxbos xxmaj xxunk ! xxmaj he matches this background colour i 'm seemingly stuck with . xxmaj thanks . ( talk ) 21:51 , xxmaj january 11 , 2016 ( xxup utc ),xxbos xxmaj hey man , i 'm really not trying to edit war . xxmaj it 's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page . xxmaj he seems to care more about the formatting than the actual info .,xxbos " 
  xxmaj more 
  i ca n't make any real suggestions on improvement - i wondered if the section statistics should be later on , or a subsection of " " types o

In [25]:
learn.loss_func

FlattenedLoss of BCEWithLogitsLoss()

In [30]:
data_class.show_batch()

text,target
xxbos xxmaj take that ! \n \n xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in,toxic;severe_toxic;obscene
xxbos i xxup am xxup an xxup loser xxrep 4 = i xxup am xxup an xxup loser = = = = i xxup am xxup an xxup loser xxrep 4 = i xxup am xxup an xxup loser = = = = i xxup am xxup an xxup loser xxrep 4 = i xxup am xxup an xxup loser = = = = i xxup am xxup an xxup,toxic
xxbos xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape \n xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup rape xxup anal xxup,toxic;severe_toxic;obscene
xxbos xxmaj why xxmaj did you delete our post about the world famous xxmaj angler xxmaj john xxmaj bailey xxrep 4 ? \n = xxmaj we manager xxmaj john xxmaj bailey and he is well known in the xxup uk population ? xxmaj we had nt even finished editing the article you nooob ! xxrep 5 = xxmaj why xxmaj did you delete our post about the world famous,
xxbos { { unblock| xxmaj oxymoron is a moron in an xxmaj oxymoron xxmaj oxymoron is a moron in an xxmaj oxymoron xxmaj oxymoron is a moron in an xxmaj oxymoron xxmaj oxymoron is a moron in an xxmaj oxymoron xxmaj oxymoron is a moron in an xxmaj oxymoron xxmaj oxymoron is a moron in an xxmaj oxymoron xxmaj oxymoron is a moron in an xxmaj oxymoron xxmaj oxymoron is,toxic


In [31]:
data_class.c

6

In [32]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,time
0,0.061105,0.060622,02:22


In [ ]:
learn.save('classifier')

In [33]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,time
0,0.046978,0.046524,06:18
1,0.048529,0.039017,05:10
2,0.033087,0.038039,06:13


In [34]:
learn.save('classifier')

In [31]:
learn.load('classifier')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (143614 items)
x: TextList
xxbos xxmaj explanation 
  xxmaj why the edits made under my username xxmaj hardcore xxmaj metallica xxmaj fan were reverted ? xxmaj they were n't vandalisms , just closure on some gas after i voted at xxmaj new xxmaj york xxmaj dolls xxup fac . xxmaj and please do n't remove the template from the talk page since i 'm retired xxunk,xxbos xxmaj xxunk ! xxmaj he matches this background colour i 'm seemingly stuck with . xxmaj thanks . ( talk ) 21:51 , xxmaj january 11 , 2016 ( xxup utc ),xxbos xxmaj hey man , i 'm really not trying to edit war . xxmaj it 's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page . xxmaj he seems to care more about the formatting than the actual info .,xxbos " 
  xxmaj more 
  i ca n't make any real suggestions on improvement - i wondered if the section statistics should be later on , or a subsection of " " types o

In [35]:
learn.predict('anal')

(MultiCategory tensor([1., 0., 1., 0., 0., 0.]),
 tensor([1., 0., 1., 0., 0., 0.]),
 tensor([9.8349e-01, 1.4393e-02, 9.4580e-01, 2.4759e-05, 1.8479e-02, 8.3238e-06]))

In [36]:
label_cols

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [37]:
learn.predict('good')

(MultiCategory tensor([0., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 0., 0., 0., 0.]),
 tensor([0.0247, 0.0012, 0.0079, 0.0002, 0.0047, 0.0005]))

In [38]:
learn.predict('sex')

(MultiCategory tensor([1., 0., 0., 0., 0., 0.]),
 tensor([1., 0., 0., 0., 0., 0.]),
 tensor([9.7119e-01, 2.5479e-03, 1.1962e-01, 5.6994e-05, 1.3107e-02, 5.9339e-03]))

In [39]:
learn.predict('child')

(MultiCategory tensor([0., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 0., 0., 0., 0.]),
 tensor([0.1712, 0.0018, 0.0081, 0.0002, 0.0180, 0.0058]))

In [40]:
learn.predict('fucking idiot')

(MultiCategory tensor([1., 1., 1., 0., 1., 0.]),
 tensor([1., 1., 1., 0., 1., 0.]),
 tensor([9.9999e-01, 9.2421e-01, 9.9996e-01, 5.2890e-07, 9.9732e-01, 3.0394e-03]))

In [43]:
! ls {path}/models

1toxic.pth  3toxic.pth	    clastoxic.pth      fine_tuned_enc_10.pth
2toxic.pth  classifier.pth  fine_tuned_10.pth  tmp.pth


## Push to kaggle

In [33]:
learn.export()

In [34]:
learn = load_learner(path, test=test)

In [35]:
preds, x = learn.get_preds(ds_type=DatasetType.Test)

In [47]:
submid = pd.DataFrame({'id': subm["id"]})

In [61]:
submission = pd.concat([submid, pd.DataFrame(preds.numpy(), columns = label_cols)], axis=1)

In [62]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.995590,0.444771,0.974723,0.089854,0.950767,0.864332
1,0000247867823ef7,0.001050,0.000205,0.000647,0.000025,0.000621,0.000158
2,00013b17ad220c46,0.001673,0.000102,0.000615,0.000006,0.000433,0.000113
3,00017563c3f7919a,0.000187,0.000042,0.000198,0.000028,0.000123,0.000024
4,00017695ad8997eb,0.006711,0.000366,0.002084,0.000144,0.001678,0.000419


In [67]:
submission.to_csv(f'{path}/submission.csv', index=False)

In [68]:
! kaggle competitions submit jigsaw-toxic-comment-classification-challenge -f {path}/submission.csv -m "fffff"

401 - Unauthorized


In [2]:
#todo: play with loss, add correct accuracy